In [0]:
%sql
SHOW EXTERNAL LOCATIONS;

In [0]:
%sql
LIST 'abfss://demo@dbxdl.dfs.core.windows.net/lineitems/' WITH (CREDENTIAL `dbxdl-storage-account-creds`);

In [0]:
%sql
SHOW CATALOGS;

In [0]:
%sql
DROP CATALOG IF EXISTS MASTERCLASS CASCADE;
CREATE CATALOG IF NOT EXISTS MASTERCLASS;

In [0]:
%sql
USE CATALOG MASTERCLASS;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS BRONZE
 MANAGED LOCATION "abfss://delta@dbxdl.dfs.core.windows.net/bronze/";

In [0]:
%sql
SELECT * FROM PARQUET.`abfss://warehouse@dbxdl.dfs.core.windows.net/lineitems/*` LIMIT 5;

In [0]:
%sql
CREATE OR REPLACE TABLE MASTERCLASS.BRONZE.RAW_LINEITEMS_WAREHOUSE (
  V VARIANT
) USING DELTA;

In [0]:
%sql
COPY INTO MASTERCLASS.BRONZE.RAW_LINEITEMS_WAREHOUSE
FROM (
  SELECT parse_json(to_json(struct(*))) AS V 
  FROM 'abfss://warehouse@dbxdl.dfs.core.windows.net/lineitems/*'
)
FILEFORMAT = PARQUET
FORMAT_OPTIONS ('singleVariantColumn' = 'true');

In [0]:
%sql
-- Counting 59,986,052
SELECT COUNT(*) FROM MASTERCLASS.BRONZE.RAW_LINEITEMS_WAREHOUSE;

In [0]:
%sql
-- Selecting 5 random values
SELECT
  variant_get(V, '$.L_ORDERKEY') AS order_key,
  variant_get(V, '$.L_PARTKEY') AS part_key,
  variant_get(V, '$.L_SUPPKEY') AS supp_key,
  variant_get(V, '$.L_LINENUMBER') AS line_number,
  variant_get(V, '$.L_QUANTITY') AS quantity,
  variant_get(V, '$.L_EXTENDEDPRICE') AS extended_price,
  variant_get(V, '$.L_DISCOUNT') AS discount,
  variant_get(V, '$.L_TAX') AS tax,
  variant_get(V, '$.L_RETURNFLAG') AS return_flag,
  variant_get(V, '$.L_LINESTATUS') AS line_status,
  variant_get(V, '$.L_SHIPDATE') AS ship_date,
  variant_get(V, '$.L_COMMITDATE') AS commit_date,
  variant_get(V, '$.L_RECEIPTDATE') AS receipt_date,
  variant_get(V, '$.L_SHIPINSTRUCT') AS ship_instruct,
  variant_get(V, '$.L_SHIPMODE') AS ship_mode,
  variant_get(V, '$.L_COMMENT') AS comment
FROM MASTERCLASS.BRONZE.RAW_LINEITEMS_WAREHOUSE
ORDER BY RAND() LIMIT 5;

In [0]:
%sql
-- QUERY RECORDS
SELECT
  variant_get(V, '$.L_ORDERKEY') AS order_key,
  variant_get(V, '$.L_PARTKEY') AS part_key,
  variant_get(V, '$.L_SUPPKEY') AS supp_key,
  variant_get(V, '$.L_LINENUMBER') AS line_number,
  variant_get(V, '$.L_QUANTITY') AS quantity,
  variant_get(V, '$.L_EXTENDEDPRICE') AS extended_price,
  variant_get(V, '$.L_DISCOUNT') AS discount,
  variant_get(V, '$.L_TAX') AS tax,
  variant_get(V, '$.L_RETURNFLAG') AS return_flag,
  variant_get(V, '$.L_LINESTATUS') AS line_status,
  variant_get(V, '$.L_SHIPDATE') AS ship_date,
  variant_get(V, '$.L_COMMITDATE') AS commit_date,
  variant_get(V, '$.L_RECEIPTDATE') AS receipt_date,
  variant_get(V, '$.L_SHIPINSTRUCT') AS ship_instruct,
  variant_get(V, '$.L_SHIPMODE') AS ship_mode,
  variant_get(V, '$.L_COMMENT') AS comment
FROM MASTERCLASS.BRONZE.RAW_LINEITEMS_WAREHOUSE
-- This will failed due to the premature SQL engine, the attribute needs to be casted twice
--WHERE order_key IN (
WHERE CAST(variant_get(V, '$.L_ORDERKEY') AS STRING) IN (
'30370724',
'43675749',
'46386755',
'39896960',
'51780611'
);

In [0]:
%sql
-- Creating table
CREATE OR REPLACE TABLE MASTERCLASS.BRONZE.LINEITEMS_WAREHOUSE
USING DELTA
AS
SELECT
  CAST(variant_get(V, '$.L_ORDERKEY') AS VARCHAR(15)) AS L_ORDERKEY,
  CAST(variant_get(V, '$.L_PARTKEY') AS VARCHAR(15)) AS L_PARTKEY,
  CAST(variant_get(V, '$.L_SUPPKEY') AS VARCHAR(15)) AS L_SUPPKEY,
  CAST(variant_get(V, '$.L_LINENUMBER') AS INT) AS L_LINENUMBER,
  CAST(variant_get(V, '$.L_QUANTITY') AS FLOAT) AS L_QUANTITY,
  CAST(variant_get(V, '$.L_EXTENDEDPRICE') AS FLOAT) AS L_EXTENDEDPRICE,
  CAST(variant_get(V, '$.L_DISCOUNT') AS FLOAT) AS L_DISCOUNT,
  CAST(variant_get(V, '$.L_TAX') AS FLOAT) AS L_TAX,
  CAST(variant_get(V, '$.L_RETURNFLAG') AS VARCHAR(30)) AS L_RETURNFLAG,
  CAST(variant_get(V, '$.L_LINESTATUS') AS VARCHAR(30)) AS L_LINESTATUS,
  CAST(variant_get(V, '$.L_SHIPDATE') AS DATE) AS L_SHIPDATE,
  CAST(variant_get(V, '$.L_COMMITDATE') AS DATE) AS L_COMMITDATE,
  CAST(variant_get(V, '$.L_RECEIPTDATE') AS DATE) AS L_RECEIPTDATE,
  CAST(variant_get(V, '$.L_SHIPINSTRUCT') AS VARCHAR(30)) AS L_SHIPINSTRUCT,
  CAST(variant_get(V, '$.L_SHIPMODE') AS VARCHAR(30)) AS L_SHIPMODE,
  CAST(variant_get(V, '$.L_COMMENT') AS VARCHAR(100)) AS L_COMMENT
FROM masterclass.bronze.raw_lineitems_warehouse;

DROP THINGS

In [0]:
%sql
USE CATALOG MASTERCLASS;
USE SCHEMA bronze;

In [0]:
%sql
DROP TABLE IF EXISTS masterclass.bronze.lineitems;
DROP TABLE IF EXISTS masterclass.bronze.lineitems_warehouse;
DROP TABLE IF EXISTS masterclass.bronze.raw_lineitems;
DROP TABLE IF EXISTS masterclass.bronze.raw_lineitems_warehouse;
DROP TABLE IF EXISTS masterclass.bronze.raw_lineitems_dlt;

In [0]:
%sql
DROP SCHEMA IF EXISTS masterclass.bronze CASCADE;
DROP SCHEMA IF EXISTS masterclass.silver CASCADE;
DROP SCHEMA IF EXISTS masterclass.default CASCADE;